In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append("./../../")

In [125]:
from pathlib import Path
import math
#
import torch
import torch.nn as nn
#
import matplotlib.pyplot as plt
import numpy as np
#

from effcn.models_mnist import EffCapsNet, CNN_CR_SF, CNN_CR, CNN_R, CapsNet, Backbone, BackboneHinton, CNN_CR_SF_Backbone, Decoder
from misc.utils import count_parameters
from effcn.layers import Squash, PrimaryCaps, FCCaps
from effcn.functions import masking

In [5]:
model = EffCapsNet()

count_parameters(model)

1573168

In [87]:
model = CapsNet()

r = count_parameters(model)
r

8227600

In [28]:
model = CNN_CR_SF()

count_parameters(model)


7941424

In [29]:
model = Backbone()

print(count_parameters(model))

x = torch.rand([1,1,28,28])
x = model(x)
x.shape

130688


torch.Size([1, 128, 9, 9])

In [129]:
model = Decoder()

print(count_parameters(model))

1411344


In [203]:
class Backbone_EffCN_HPar(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=512,
                kernel_size=9,
                stride=1,
                padding=0,
            ),
            nn.ReLU(),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(512, 512, 3, 1),
            nn.ReLU(),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(512, 256, 5, 1),
            nn.ReLU(),
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(256, 128, 6, 1),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)

        return x

In [206]:
model = Backbone_EffCN_HPar()

print(count_parameters(model))

x = torch.rand([1,1,28,28])
x = model(x)
print(x.shape)


print(r-count_parameters(model)-count_parameters(Decoder()))
print(r / (count_parameters(model)+count_parameters(Decoder())))

6858624
torch.Size([1, 128, 9, 9])
-42368
0.9948768846505814


In [210]:
class EffCapsNet_HPar(nn.Module):

    def __init__(self):
        super().__init__()
        # values from paper, are fixed!
        self.n_l = 16  # num of primary capsules
        self.d_l = 8   # dim of primary capsules
        self.n_h = 10  # num of output capsules
        self.d_h = 16  # dim of output capsules

        self.backbone = Backbone_EffCN_HPar()
        self.primcaps = PrimaryCaps(
            F=128, K=9, N=self.n_l, D=self.d_l)  # F = n_l * d_l !!!
        self.fcncaps = FCCaps(self.n_l, self.n_h, self.d_l, self.d_h)
        self.decoder = Decoder()

    def forward(self,x, y_true=None):
        """
            IN:
                x (b, 1, 28, 28)
            OUT:
                u_h
                    (b, n_h, d_h)
                    output caps
                x_rec
                    (b, 1, 28, 28)
                    reconstruction of x
        """
        u_l = self.primcaps(self.backbone(x))
        u_h = self.fcncaps(u_l)
        #
        u_h_masked = masking(u_h, y_true)
        x_rec = self.decoder(u_h_masked)
        return u_h, x_rec

In [211]:
model = EffCapsNet_HPar()

print(count_parameters(model))

x = torch.rand([1,1,28,28])
x, a = model(x)
print(x.shape)


print(r-count_parameters(model))
print(r / count_parameters(model))

8301104
torch.Size([1, 10, 16])
-73504
0.9911452741707609
